In [1]:
#Import tools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from config import api_key
from config import gkey
import gmaps
import os
import json

In [2]:
#read csv
weather_df = pd.read_csv("../Weather.py/output/cities_final.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,date
0,Mahon,20,ES,1612055710,93,39.8885,4.2658,48.20,5.75,2021-01-31 01:15:10
1,Baracoa,100,CU,1612055710,80,20.3467,-74.4958,71.98,13.60,2021-01-31 01:15:10
2,Saskylakh,100,RU,1612055710,82,71.9167,114.0833,-33.81,11.70,2021-01-31 01:15:10
3,Thompson,90,CA,1612055698,92,55.7435,-97.8558,14.00,2.89,2021-01-31 01:14:58
4,Korla,0,CN,1612055711,85,41.7597,86.1469,18.37,4.43,2021-01-31 01:15:11
...,...,...,...,...,...,...,...,...,...,...
767,Mugumu,50,TZ,1612058486,88,-1.8500,34.7000,58.06,4.74,2021-01-31 02:01:26
768,Eureka,90,US,1612058487,87,40.8021,-124.1637,53.01,4.61,2021-01-31 02:01:27
769,Olonets,100,RU,1612058487,98,60.9847,32.9698,20.17,4.92,2021-01-31 02:01:27
770,Sahuaripa,0,MX,1612058487,39,29.0500,-109.2333,60.30,1.34,2021-01-31 02:01:27


In [3]:
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations abd humidity in a humidity variable
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2)

# Add heat layer
fig.add_layer(heat_layer)

#show fig
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Find cities using given parameters
city_df = weather_df.loc[(weather_df["Max Temp"] < 80) \
                                & (weather_df["Max Temp"] > 70) \
                                & (weather_df["Wind Speed"] < 10) \
                                & (weather_df["Cloudiness"] == 0)].dropna() 

In [6]:
#Add Hotel Name column
city_df['Hotel Name'] = ""
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,date,Hotel Name
389,Hua Hin,0,TH,1612058368,73,12.5707,99.9588,75.20,3.44,2021-01-31 01:59:28,
390,Fonte Boa,0,BR,1612058369,89,-2.4959,-66.2662,75.63,2.73,2021-01-31 01:59:29,
496,Saint-Philippe,0,RE,1612058393,65,-21.3585,55.7679,78.80,3.44,2021-01-31 01:59:53,
613,Caravelas,0,BR,1612058440,87,-17.7125,-39.2481,75.24,5.23,2021-01-31 02:00:40,
630,Navrongo,0,GH,1612058445,20,10.8956,-1.0921,73.08,8.21,2021-01-31 02:00:45,
631,Ahipara,0,NZ,1612058445,51,-35.1667,173.1667,71.69,5.64,2021-01-31 02:00:45,
660,Ouargaye,0,BF,1612058455,20,11.5033,0.0572,71.83,9.28,2021-01-31 02:00:55,
693,Tomatlán,0,MX,1612058467,66,19.9333,-105.2500,71.22,4.09,2021-01-31 02:01:07,
753,Ambilobe,0,MG,1612058483,93,-13.2000,49.0500,72.32,2.21,2021-01-31 02:01:23,


In [7]:
#Show only the values we need
hotel_df = city_df[['Hotel Name','City','Country','Lat','Lng']]

#ser base url and parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [8]:
#Create for loop
for index, row in hotel_df.iterrows(): 
    #grab the longitude, latitude, city and country from city_df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    Country = row["Country"]
    
    #Add location to the parameter dictionary
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    
    response = requests.get(base_url,params=params).json()
    
    #create a variable with the results
    results = response['results']
    
    try:
        #Add Hotel Name to hotel_df
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']
        print(f"{results[0]['name']} is the closest hotel to {city}")

    except (KeyError, IndexError):
            print("Missing information")


C:\Users\rstro\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


InterContinental is the closest hotel to Hua Hin
Missing information
Les Embruns Du Baril is the closest hotel to Saint-Philippe
Mar Doce de Abrolhos is the closest hotel to Caravelas
Azakka Hotel is the closest hotel to Navrongo
Ahipara Bay Motel is the closest hotel to Ahipara
Missing information
Hotel Hacienda Vieja is the closest hotel to Tomatlán
Hotel National is the closest hotel to Ambilobe


In [11]:
#drop the rows that are missing information
hotel_df = (hotel_df.drop(hotel_df.index[[1, 6]]))
hotel_df

,Hotel Name,City,Country,Lat,Lng
389,InterContinental,Hua Hin,TH,12.5707,99.9588
496,Les Embruns Du Baril,Saint-Philippe,RE,-21.3585,55.7679
613,Mar Doce de Abrolhos,Caravelas,BR,-17.7125,-39.2481
630,Azakka Hotel,Navrongo,GH,10.8956,-1.0921
631,Ahipara Bay Motel,Ahipara,NZ,-35.1667,173.1667
693,Hotel Hacienda Vieja,Tomatlán,MX,19.9333,-105.2500
753,Hotel National,Ambilobe,MG,-13.2000,49.0500


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))